In [190]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering,Birch,OPTICS
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import matplotlib.pyplot as plt
import pyodbc
import os

In [191]:
# Connection String
serverName = "datasolve.database.windows.net" 
databaseName = "DataSolveDB"
username = "sqladmin"
password = "AlfaBravo2024"

In [192]:
# Make the conncection to our Server

connection_string = f'DRIVER={{SQL Server}};SERVER='+serverName+';DATABASE='+databaseName+';UID='+username+';PWD='+ password

cnxn = pyodbc.connect(connection_string)
cursor = cnxn.cursor()

In [193]:
df = pd.read_sql_query("select * from ml.Death_GHDX",con=cnxn)
df

C:\Users\IreneDeNevi\AppData\Local\Temp\ipykernel_22476\3213978097.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("select * from ml.Death_GHDX",con=cnxn)


,cluster_sdi,Risk,Val,location,year,SDI
0,Low,Tobacco,0.245127,China,1990,0.433
1,Low,Alcohol use,0.102353,China,1990,0.433
2,Low,Metabolic risks,0.484631,China,1990,0.433
3,Low,Dietary risks,0.144456,China,1990,0.433
4,Low,Low physical activity,0.045454,China,1990,0.433
...,...,...,...,...,...,...
30895,Low,Tobacco,0.163817,Sudan,2019,0.515
30896,Low,Alcohol use,0.002127,Sudan,2019,0.515
30897,Low,Metabolic risks,0.712675,Sudan,2019,0.515
30898,Low,Dietary risks,0.107153,Sudan,2019,0.515


In [194]:
df.shape

(30900, 6)

In [197]:
categorical_features = ['location','Risk','cluster_sdi']
numerical_features = []

In [198]:
label_encoder = LabelEncoder()
for feature in categorical_features:
    df[feature] = label_encoder.fit_transform(df[feature])

# Extract fe1atures for clustering
X = df[numerical_features + categorical_features ]

# Standardize the numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [199]:
#Function to perform Clustering and scores
def perform_clustering(cluster_model, X):
    labels = cluster_model.fit_predict(X)
    df['Cluster'] = labels
    silhouette_score_value = silhouette_score(X, labels)
    davies_bouldin_index = davies_bouldin_score(X, labels)
    calinski_harabasz_index = calinski_harabasz_score(X, labels)
    return silhouette_score_value, davies_bouldin_index, calinski_harabasz_index, labels

In [200]:
# Function to perform PCA for dimensionality reduction
def perform_pca(X):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    return X_pca    # visualizing high-dimensional data in a 2D space

In [201]:
# Function to plot clusters
def plot_clusters(X, labels, title, method_name):
    unique_labels = np.unique(labels)
    scatter = plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', edgecolors='k', s=50)
    
    # Create a legend
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=scatter.cmap(scatter.norm(label)), markersize=8, label=f'Cluster {label}') for label in unique_labels]
    plt.legend(handles=handles, title='Clusters', loc='upper right')
    
    plt.title(f'{method_name} Clustering')
    

    return plt.gcf()

In [202]:
# Dictionary of clustering methods
clustering_methods = {
    'K-means': KMeans(n_clusters=5, random_state=42, n_init="auto"),
    'Hierarchical': AgglomerativeClustering(n_clusters=5, metric='euclidean'),
    'DBSCAN': DBSCAN(eps=0.5, min_samples=5, metric='euclidean', metric_params=None, algorithm='auto', leaf_size=30, p=None, n_jobs=None),
    'Birch': Birch(),
    'OPTICS': OPTICS(min_samples=5, xi=0.05)
}

In [203]:
cluster_scores = {'Method': [], 'Silhouette Score': [], 'Davies-Bouldin Index': [], 'Calinski-Harabasz Index': []}


if not os.path.exists('Plots'):
    os.makedirs('Plots')

for method_name, cluster_model in clustering_methods.items():
    silhouette_score_value, davies_bouldin_index, calinski_harabasz_index, labels = perform_clustering(cluster_model, X_scaled)
    
    categorical_info = ', '.join(categorical_features)
    numerical_info = ', '.join(numerical_features)
    title = f'{method_name} Clustering\nCategorical Features: {categorical_info}\nNumerical Features: {numerical_info}'
    fig = plot_clusters(perform_pca(X_scaled), labels, title, method_name)
    fig.savefig(f'Plots/{method_name}_{categorical_info}_{numerical_info}_Clustering_Plot.png')
    plt.close(fig)
    
    # Save scores
    cluster_scores['Method'].append(method_name)
    cluster_scores['Silhouette Score'].append(silhouette_score_value)
    cluster_scores['Davies-Bouldin Index'].append(davies_bouldin_index)
    cluster_scores['Calinski-Harabasz Index'].append(calinski_harabasz_index)

C:\Users\IreneDeNevi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_optics.py:932: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [204]:
#Save the results

scores_df = pd.DataFrame(cluster_scores)
current_directory = os.getcwd()

categorical_names = '_'.join(categorical_features)
numerical_names = '_'.join(numerical_features)
csv_file_name = f'scoresCluster_{categorical_names}_{numerical_names}.csv'


score_folder = os.path.join(current_directory, 'ScoreCSV')
if not os.path.exists(score_folder):
    os.makedirs(score_folder)

csv_file_path = os.path.join(score_folder, csv_file_name)
scores_df.to_csv(csv_file_path, index=False)


In [205]:
cnxn.close()